In [6]:
import torch
import transformers
from transformers import AutoTokenizer
name = 'dfurman/mpt-7b-instruct-orca'

config = transformers.AutoConfig.from_pretrained(name, trust_remote_code=True)
config.attn_config['attn_impl'] = 'triton'
config.init_device = 'meta' # For fast initialization

model = transformers.AutoModelForCausalLM.from_pretrained(
  name,
  config=config,
  torch_dtype=torch.bfloat16, # Load model weights in bfloat16
  trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")


Instantiating an MPTForCausalLM model from /home/paperspace/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b/72e5f594ce36f9cabfa2a9fd8f58b491eb467ee7/modeling_mpt.py


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  8.20it/s]


In [7]:
from transformers import pipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device= device)
with torch.autocast('cuda', dtype=torch.bfloat16):
    print(
        pipe('Here is a recipe for vegan banana bread:\n',
            max_new_tokens=100,
            do_sample=True,
            use_cache=True))

NotImplementedError: Cannot copy out of meta tensor; no data!